In [1]:
import http.client, urllib, base64, requests
import cognitive_face as CF 
from cognitive_face import util
import pandas as pd
import time

In [2]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

/Users/Amiros/anaconda/lib/python3.5/site-packages/IPython/core/magics/extension.py:47: UserWarning: %install_ext` is deprecated, please distribute your extension as a python package.
  "as a python package.", UserWarning)


Installed autotime.py. To use it, type:
  %load_ext autotime


In [3]:
#Emotion Analysis parameters
_url = 'https://api.projectoxford.ai/emotion/v1.0/recognize'
_key = '183aea69820e4dfda0176db816fc1f72' #Here you have to paste your primary key
_maxNumRetries = 5

#Face detection parametrs
CF.Key.set('d78211e12bba41a991382ade5d68745b') #set the key

time: 2.83 ms


In [22]:
#Get all the frames from S3
url_list = []
for i in range(4920,5920,5):
    url = "https://s3-us-west-2.amazonaws.com/debateinemotion/debate1/frame%d.jpg" % i
    url_list.append(url)

# Image path
urlImage= ''

time: 3.4 ms


In [5]:
# Emtional Anlaysis function
def processRequest( json, data, headers, params ):

    retries = 0
    result = None

    while True:

        response = requests.request( 'post', _url, json = json, data = data, headers = headers, params = params )

        if response.status_code == 429: 

            print( "Message: %s" % ( response.json()['error']['message'] ) )

            if retries <= _maxNumRetries: 
                time.sleep(1) 
                retries += 1
                continue
            else: 
                print( 'Error: failed after retrying!' )
                break

        elif response.status_code == 200 or response.status_code == 201:

            if 'content-length' in response.headers and int(response.headers['content-length']) == 0: 
                result = None 
            elif 'content-type' in response.headers and isinstance(response.headers['content-type'], str): 
                if 'application/json' in response.headers['content-type'].lower(): 
                    result = response.json() if response.content else None 
                elif 'image' in response.headers['content-type'].lower(): 
                    result = response.content
        else:
            print( "Error code: %d" % ( response.status_code ) )
            print( "Message: %s" % ( response.json()['error']['message'] ) )

        break
        
    return result


headers = dict()
headers['Ocp-Apim-Subscription-Key'] = _key
headers['Content-Type'] = 'application/json' 

json = { 'url': urlImage } 
data = None
params = None

time: 42.7 ms


In [6]:
# Face detect function
def detect(image, face_id=True, landmarks=False, faceRectangle= False, attributes='gender'):
    
    url = 'detect'
    headers, data, json = util.parse_image(image)
    params = {
        'returnFaceId': face_id and 'true',
        'returnFaceLandmarks': landmarks and 'false',
        'returnFaceAttributes': attributes,
    }

    return util.request('POST', url, headers=headers, params=params, json=json,
                        data=data)

time: 6.06 ms


In [ ]:
# Face Detection & Emotioin Analysis 
#emotions = []
count = 0

for single_image in url_list:
    urlImage = single_image
    json = { 'url': urlImage } 
    emotions.append(detect(single_image))
    emotions.append(int(count))
    emotions[count].append(processRequest(json, data, headers, params))
    count += 1
    time.sleep(3.5)

In [21]:
len(emotions)

913

time: 3.7 ms


In [ ]:
#Turning Json to pandas dataframe

#function to flatten json objects
def flattenjson( b, delim ):
    val = {}
    for i in b.keys():
        if isinstance( b[i], dict ):
            get = flattenjson( b[i], delim )
            for j in get.keys():
                val[ i + delim + j ] = get[j]
        else:
            val[i] = b[i]

    return val

In [ ]:
#select keys that are needed for analysis - it can be modified as needed
wanted_keys = ['faceAttributes', 'faceId', 'scores']

In [ ]:
f = []
for index, entry in enumerate(final_data):
    flat_dict = flattenjson(final_data[index], ",")
    trimmed_dict = dict((k, flat_dict[k]) for k in wanted_keys if k in flat_dict)
    f.append(trimmed_dict)

In [ ]:
pd.DataFrame(final_data)